## Imports

In [1]:
import pandas as pd
import itertools
from gurobipy import *

## Helper functions

In [2]:
def readData():
    xls = pd.ExcelFile("data.xlsx")
    
    # Getting point coordinates
    sheet = xls.parse(0) 
    
    xCoordinates = list(sheet['X-coordinate'])
    yCoordinates = list(sheet['Y-coordinate'])
    points = [(xCoordinates[i], yCoordinates[i]) for i in range(len(xCoordinates))]
    
    # Getting block data
    sheet = xls.parse(1, skiprows = [0])
    
    xCoordinates = list(sheet['X coordinate'])
    yCoordinates = list(sheet['Y coordinate'])
    widths = list(sheet['Width'])
    heights = list(sheet['Height'])
    blocks = [(xCoordinates[i], yCoordinates[i], widths[i], heights[i]) for i in range(len(xCoordinates))]
    
    return points, blocks

In [3]:
def manhattanDistance(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    
    return abs(x1 - x2) + abs(y1 - y2)

In [4]:
def calculateDistances(points):
    Cij = []
    for i in range(0, len(points)):
        Cj = []
        for j in range(0, len(points)):
            Cj.append(manhattanDistance(points[i], points[j]))
        Cij.append(Cj)

    return Cij

## Parameters

In [5]:
points, blocks = readData()
N = len(points)
C = calculateDistances(points)

## Creating model

In [6]:
model = Model()

Using license file /Users/fuadaghazada/gurobi.lic
Academic license - for non-commercial use only


## Decision Variables

In [7]:
x = {}
for i in range(N):
    for j in range(N):
        x[i, j] = model.addVar(vtype = GRB.BINARY, name = f"X_{i}_{j}")
    model.update()
    
u = {}
for i in range(N):
    u[i] = model.addVar(vtype = GRB.CONTINUOUS, name = f"U_{i}")

## Constraints

In [8]:
for i in range(N):    
    model.addConstr(sum(x[i, j] for j in range(N)), GRB.EQUAL, 1)
    
model.update()

In [9]:
for j in range(N):
    model.addConstr(sum(x[i, j] for i in range(N)), GRB.EQUAL, 1)
    
model.update()

In [10]:
for i in range(N):
    model.addConstr(x[i, i], GRB.EQUAL, 0)
    
model.update()

In [11]:
for i in range(1, N):
    for j in range(1, N):
        model.addConstr(u[i] - u[j] + (N - 1) * x[i, j], GRB.LESS_EQUAL, N - 2)

model.update()

## Objective function

In [12]:
obj = 0
for i in range(N):
    for j in range(N):
        obj += C[i][j] * x[i, j] 

In [13]:
model.update()
model.setObjective(obj, GRB.MINIMIZE)
model.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 2551 rows, 2550 columns and 12155 nonzeros
Model fingerprint: 0x59a9ab76
Variable types: 50 continuous, 2500 integer (2500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [1e+00, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Presolve removed 99 rows and 51 columns
Presolve time: 0.02s
Presolved: 2452 rows, 2499 columns, 11956 nonzeros
Variable types: 49 continuous, 2450 integer (2450 binary)

Root relaxation: objective 2.978776e+02, 158 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  297.87755    0   84          -  297.87755      -     -    0s
     0     0  346.10204    0  101          -  346.10204      -     -    0s
     0     0  348.04082    0  108          -  348.04082      -     -    0s
     0     0  

In [15]:
for v in model.getVars():
    if v.X != 0:
        print("%s %f" % (v.Varname, v.X))

X_0_47 1.000000
X_1_22 1.000000
X_2_9 1.000000
X_3_42 1.000000
X_4_46 1.000000
X_5_20 1.000000
X_6_40 1.000000
X_7_37 1.000000
X_8_7 1.000000
X_9_43 1.000000
X_10_21 1.000000
X_11_10 1.000000
X_12_23 1.000000
X_13_0 1.000000
X_14_28 1.000000
X_15_6 1.000000
X_16_1 1.000000
X_17_25 1.000000
X_18_3 1.000000
X_19_49 1.000000
X_20_30 1.000000
X_21_17 1.000000
X_22_8 1.000000
X_23_35 1.000000
X_24_16 1.000000
X_25_34 1.000000
X_26_15 1.000000
X_27_4 1.000000
X_28_39 1.000000
X_29_41 1.000000
X_30_14 1.000000
X_31_33 1.000000
X_32_2 1.000000
X_33_38 1.000000
X_34_13 1.000000
X_35_24 1.000000
X_36_5 1.000000
X_37_18 1.000000
X_38_32 1.000000
X_39_12 1.000000
X_40_36 1.000000
X_41_11 1.000000
X_42_29 1.000000
X_43_48 1.000000
X_44_45 1.000000
X_45_31 1.000000
X_46_19 1.000000
X_47_27 1.000000
X_48_26 1.000000
X_49_44 1.000000
U_1 32.000000
U_2 12.000000
U_3 38.000000
U_4 2.000000
U_5 21.000000
U_6 18.000000
U_7 35.000000
U_8 34.000000
U_9 13.000000
U_10 43.000000
U_11 42.000000
U_12 27.000000
